Earth Surface Heat Flow	ene.006	http://onlinelibrary.wiley.com/doi/10.1002/ggge.20271/full

ene.006 http://onlinelibrary.wiley.com/doi/10.1002/ggge.20271/full
* Read this paper, informative on spatial interpolation and reason for showing on a certain grid
Files: stored in RW_Data/Rasters/Earth Heat Flow
* Read paper for a description of which to upload
Equal area csvs: 
* ggge20271-sup-0001-Data_Table1_Eq_area_Global_heat_flow.csv
* ggge20271-sup-0005-Data_Table2_Eq_area_Data_only_HF.csv
* ggge20271-sup-0006-Data_Table3_Eq_area_Young_ocean_only_HF.csv
* ggge20271-sup-0007-Data_Table4_Eq_area_Geology_based_HF.csv
* ggge20271-sup-0008-Data_Table5_Eq_area_Data_only_detailed.csv
* ggge20271-sup-0009-DataTable6_2deg_equal_area_grid_description.csv (tells four corners of each grid cell)
Equal lat lon csvs: 
* ggge20271-sup-0003-Data_Table1_Eq_lon_lat_Global_HF.csv

Also a Heat Flow shapefile

Import libraries

In [3]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

In [4]:
pd.read_csv("/Users/nathansuberi/Desktop/RW_Data/Rasters/Earth Heat Flow/ggge20271-sup-0001-Data_Table1_Eq_area_Global_heat_flow.csv")

,Longitude,Latitude,Mean_HF,Median_HF,Total_Erro
0,-120.000,-89.0,100.0,100.0,41.50
1,0.000,-89.0,65.0,65.0,41.50
2,120.000,-89.0,65.0,65.0,41.50
3,-160.000,-87.0,100.0,100.0,41.50
4,-120.000,-87.0,100.0,100.0,41.50
5,-80.000,-87.0,100.0,100.0,41.50
6,-40.000,-87.0,100.0,100.0,41.50
7,0.000,-87.0,65.0,65.0,41.50
8,40.000,-87.0,65.0,65.0,41.50
9,80.000,-87.0,65.0,65.0,41.50


In [ ]:
# Have to convert this csv into a raster

s3 tools

In [0]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/*"

s3_file = "*.tif"

s3_key_orig = s3_folder + s3_file
s3_key_edit = s3_key_orig[0:-4] + "_edit.tif"


class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

Potentially useful for online directories

In [0]:
# View files in a ftp, see cli.015
remote_path = "ftp://cidportal.jrc.ec.europa.eu/jrc-opendata/EDGAR/datasets/v431_v2"
remote_path_BC = remote_path + "/BC/TOTALS/"
file = req.urlopen(remote_path_BC).read().splitlines()

# Copy from https or ftp
with(closing(urlopen(online_folder + most_recent))) as r:
    with(open(local_orig, 'wb')) as f:
        shutil.copyfileobj(r, f)

Define local file locations

In [0]:
local_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/*"
file_name = "*"]

local_orig = local_folder + file_name

orig_extension_length = 4 #4 for each char in .tif
local_edit = local_orig[:-orig_extension_length] + "edit.tif" 

Use rasterio to reproject and compress

In [0]:
# Note - this is the core of Vizz's netcdf2tif function

with rio.open(local_orig, 'r') as src:
    # This assumes data is readable by rasterio
    # May need to open instead with netcdf4.Dataset, for example
   
    data = src.read()[0]
    
    rows = data.shape[0]
    columns = data.shape[1]
    
    print(rows)
    print(columns)
    
    # Latitude bounds
    south_lat = -90
    north_lat = 90

    # Longitude bounds
    west_lon = -180
    east_lon = 180
    
    transform = rasterio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)
    
    # Profile
    
    no_data_val = *
    target_projection = 'EPSG:4326'
    target_data_type = np.float64
    
    profile = {
        'driver':'GTiff', 
        'height':rows, 
        'width':columns, 
        'count':1, 
        'dtype':target_data_type, 
        'crs':target_projection, 
        'transform':transform, 
        'compress':'lzw', 
        'nodata': no_data_val
    }
    
    with rio.open(local_edit, "w", **profile) as dst:
        dst.write(data.astype(profile["dtype"]), 1)

Upload orig and edit files to s3

In [0]:
# Original
s3_upload.upload_file(local_orig, s3_bucket, s3_key_orig,
                         Callback=ProgressPercentage(local_orig))

# Edit
s3_upload.upload_file(local_edit, s3_bucket, s3_key_edit,
                         Callback=ProgressPercentage(local_edit))